In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: pandas
Azure region: eastus2
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-205982


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException



# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

from azureml.core.compute import ComputeTarget, AmlCompute


# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [33]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform, quniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "C": uniform(0, 10),
        "max_iter": quniform(20, 180, 1)
    }
)### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.2, evaluation_interval=1, delay_evaluation=5)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./',
                            script='train.py',
                            arguments=['--C', 1.0, '--max_iter', 100],
                            compute_target=cpu_cluster,
                            )

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)### YOUR CODE HERE ###

In [22]:
%pwd

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/pandas-compute/code/Users/odl_user_205982/ml_engineering_azure-udacity-'

In [34]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
print(hyperdrive_run)
RunDetails(hyperdrive_run).show()

Run(Experiment: udacity-project,
Id: HD_f9909eb6-498b-415b-9501-91c9196617bd,
Type: hyperdrive,
Status: Running)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [15]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [35]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
#best_run.get_details()
hyperdrive_run.get_children_sorted_by_primary_metric(top=1)


### YOUR CODE HERE ###

[]

In [38]:
hyperdrive_run.get_children_sorted_by_primary_metric(top=10)

[{'run_id': 'HD_f9909eb6-498b-415b-9501-91c9196617bd_1',
  'hyperparameters': '{"C": 1.84644130019621, "max_iter": 112.0}',
  'best_primary_metric': None,
  'status': 'Failed'},
 {'run_id': 'HD_f9909eb6-498b-415b-9501-91c9196617bd_2',
  'hyperparameters': '{"C": 0.12997731443840754, "max_iter": 30.0}',
  'best_primary_metric': None,
  'status': 'Failed'},
 {'run_id': 'HD_f9909eb6-498b-415b-9501-91c9196617bd_0',
  'hyperparameters': '{"C": 1.9954819726240947, "max_iter": 116.0}',
  'best_primary_metric': None,
  'status': 'Failed'},
 {'run_id': 'HD_f9909eb6-498b-415b-9501-91c9196617bd_3',
  'hyperparameters': '{"C": 6.114075435215631, "max_iter": 70.0}',
  'best_primary_metric': None,
  'status': 'Failed'}]

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###